Augmentation with text is not used as frequently as in video computing. It is difficult with low-resource languages b/c they are not as diverse and don't have the vector space overlap with higher-resource languages. The goal below was to follow an example in an arvix paper (Data Augmentation for Low-Resource Neural Machine Translation, https://arxiv.org/pdf/1705.00440.pdf) which employed a method similar to that used in visual computing. They called it 'translation data augmentation' (TDA). It augments the training data by altering existing sentences in the parallel corpus, similar in spirit to the data augmentation approaches in computer vision. This study proposed a weaker notion of label preservation that allows to alter both source and target sentences at the same time as long as they remain translation of each other. The paper augmented only low-frequency words; the code below includes the entire set.

In [528]:
!pip install googletrans==3.1.0a0
!pip install nlpaug
!pip install transformers
pd.set_option('display.max_colwidth', None)

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.


In [529]:
import googletrans
import nlpaug
import nlpaug.augmenter.word as naw
from googletrans import Translator
import pandas as pd
import numpy as np
import re
print(googletrans.LANGUAGES)

{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

In [530]:
translator = Translator()

In [531]:
# Training file from Github shared page
df = pd.read_csv('./train.csv')  

In [663]:
# Retrieve all the text with labels == 'CULTURE'
label = df.loc[df['Label'] == 'RELIGION']

In [664]:
label

,ID,Text,Label
6,ID_ASbcNLkZ,"Kwaya ya St. James Chilomoni Yakwanitsa Zaka 40 Wolemba: Sylvester Kasitomu wp-content/uploads/2019/09/chilomoni.jpg"" alt="""" width=""427"" height=""391"" />Tchalitchi la St. James Chilomoni ku Blantyre Kwaya ya St. James ya m`parish ya Chilomoni mu arkdayosizi ya Blantyre yayamikira mamembala ake omwe akhala akutumikira mu kwayayi kwa nthawi yaitali kamba ka kakudzipereka kwao potukula maimbidwe mu kwayayi. \nWapampando wa kwayayi a Ezekiel Lokote ndiwo alankhula izi pa mwambo wa msembe ya ukaristia yothokoza mulungu kuti kwayayi yakwanitsa zaka makumi anayi (40) chiyikhazikitsireni. \nIwo ati mamembalawa achita zazikulu mu kwayayi kwa nthawi yaitali zomwe ati zathandiza kwambiri potuluka mayimbidwe a kwayayi. \nIfeyo tinachiona cha nzeru kuti tikonze mwambowu pofuna kuwathokoza mamemba athu kaamba ka ntchito yomwe agwira poonetsetsa kuti mayimbidwe mukwayayi akhalepo kwa zaka 40 zapitazi ena mwa mamembalawa akwanitsa zaka 30 akuimbira mulungu mukwayayi zomwe ndizonyaditsa Polankhulapo mmodzi mwa omwe atumikira kwayayi kwa zaka zochuluka mayi Betti Banda anati iwo anayamba kwayayi ali ndi zaka zochepa mpaka pano zomwe zachititsa kuti moyo wawo wamapempero usinthike. \nNdinayamba kwaya imeneyi ndiri ndi zaka 10 ndipo ndaimba kwayayi kwa zaka makumi atatu zomwe ndizopindulitsa mu moyo wanga wa uzimu kaamba koti pamene kwayayi ikukula inenso ndikukula muuzimu, anatero mayi Banda. \nKwaya St. James yomwe inayimbira misa yolandira mtsogoleri wa mpingo wakatolika pa dziko lonse papa yohane Paulo wachiwiri yemwe anayendera dziko lino mu chaka cha 1989 inakhazikitsidwa mu chaka cha 1979. \n",RELIGION
12,ID_AkcmOxSZ,"Dziko la Ghana Latsekula Malo Opemphelera President wa dziko la Ghana Nana Akufo-Adodo watsekula malo opempherera omwe anatsekedwa pofuna kupewa kupatsirana kwa kachilombo koyambitsa nthenda ya COVID-19 mdzikolo. \nWalengeza za nkhaniyi- Akufo-Addo Malipoti a wailesi ya BBC ati malowa ndi monga matchalitchi komanso mizikiti ndipo anthu osaposa 100 ndi omwe akuloredwa kumasonkhana mu nthawi yosapitilira ola limodzi. \nAnthu omwe akusonkhana mmalo opemphelerawa apemphedwa kupitiliza kutsatira njira zopewera kupatsirana matendawa monga kukhala motalikirana komanso kuvala chotchinga kukamwa ndi mphuno (Mask) pomwe akupemphera. \nMwazina presidenti-yu wati kuwonjezera apo ophunzira a mchaka chomaliza ayambanso kupitiriza maphunziro awo ndipo mzipata zolowera mdzikolo zikhalabe zotseka. \nGanizoli ladza pomwe chiwerengero cha anthu opezeka ndi nthendayi chaima pa 8,060 omwe ndi kuphatikizapo 36 omwe amwalira ndi nthendayi. \n",RELIGION
23,ID_BOfHSumz,"Bishop Tambala Apempha Akhristu Asamalire Atumiki Wolemba: Thokozani Chapola ontent/uploads/2019/11/bishop-tambala.jpg"" alt="""" width=""310"" height=""362"" />Tambala: Asisteri amasowa kusamalidwa komanso chitetezo Episkopi wa dayosizi ya Zomba ambuye George Desmond Tambala loweruka wapempha akhristu kuti aziwonetsa chikondi posamalira atumiki mu mpingowu. \nIwo amalankhula izi ku Domasi parish pa mwambo wa misa yotsegulira nyumba ya sisteri a chipani cha Our Lady Of Mercy chomwe achikhazikitsa kumene mu dayosiziyo. \nIwo ati asisteriwa afika ku parishiyi kuti adzatumikire choncho akhristu nawo akuyenera kuwasamalira mu njira zosiyanasiyana. \nTisangowona kuti asisteri abwera kuzatitumikira koma monga anachitira Abraham, asisteri amasowa kusamalidwa monga chitetezo chawo komanso kuwathandiza kuti ntchito yawo agwire bwinobwino, anatero Ambuye Tambala. \nAmbuye Tambala ati ndi okondwa ndi kufika kwa asisteriwa mu dayosiziyi kaamba koti athandiza kupeputsa ntchito yaikulu yomwe ili mu dayosiziyi. \nNdine okondwa kwambiri chifukwa linali khumbo lathu komanso akhristu kwa nthawi yaitali kuti tikhale ndi asisteri chifukwa ntchito ya mmunda mwa Ambuye yakula ansembe sakanatha pawokha kufikira anthu onse komanso utumiki womwe abweretsa kuno ndikuwona kuti ndi wofunika kwambiri, anatero Ambuye Tambala. \nChipanichi chili ndi likulu lake mdziko la Italy ko

In [665]:
# Use Google Translate for Chichewa to English translation
label_english_list = []
for i in range(0, len(label)):
    label_english = translator.translate(label.iloc[i]['Text'], src = 'ny', dest = 'en')
    label_english_list.append(label_english.text)

In [666]:
# Break each row into individual sentences for NLPAug
label_sentences = []
for element in label_english_list:
    # print(element)
    sentences = re.split('[\.\?\!]\s*', element)
    label_sentences.append(sentences)

In [667]:
# Flatten the list for the Augment function
label_sentences = [item for sublist in label_sentences for item in sublist]

In [668]:
# label_sentences
# del label_sentences[9:10]
label_sentences

['St',
 "Mary's Choir James Chilomoni Celebrates 40 Years Writer: Sylvester Customer wp-content / uploads / 2019/09 / chilomoni",
 'jpg "alt =" "width =" 427 "height =" 391 "/> St',
 'James Chilomoni Church in Blantyre St',
 "John's Choir James of the Lormon parish in the Archdiocese of Blantyre has commended his long-serving members of the choir for their dedicated work in promoting the choir",
 'The choir chairman, Ezekiel Lokote, made the remarks at a Eucharistic celebration to thank the choir for its 40th anniversary',
 'They say that the members have done great things in the choir for a long time which they have done a lot to get out of the choir',
 'We thought it was a good idea to organize the event to thank our members for their work in making the choir live for the last 40 years',
 'started the choir at a young age so far that has changed their religious life',
 'I started this choir when I was 10 years old and I sang in the choir for 30 years which is very rewarding in my spi

In [669]:
# A Dataframe to hold the Augmented Sentences and Translations to Chichewa
augment_sents = pd.DataFrame()

In [670]:
augment_sents.insert(0, 'orig sent', label_sentences)

In [671]:
augment_sents.head(10)

,orig sent
0,St
1,Mary's Choir James Chilomoni Celebrates 40 Years Writer: Sylvester Customer wp-content / uploads / 2019/09 / chilomoni
2,"jpg ""alt ="" ""width ="" 427 ""height ="" 391 ""/> St"
3,James Chilomoni Church in Blantyre St
4,John's Choir James of the Lormon parish in the Archdiocese of Blantyre has commended his long-serving members of the choir for their dedicated work in promoting the choir
5,"The choir chairman, Ezekiel Lokote, made the remarks at a Eucharistic celebration to thank the choir for its 40th anniversary"
6,They say that the members have done great things in the choir for a long time which they have done a lot to get out of the choir
7,We thought it was a good idea to organize the event to thank our members for their work in making the choir live for the last 40 years
8,started the choir at a young age so far that has changed their religious life
9,"I started this choir when I was 10 years old and I sang in the choir for 30 years which is very rewarding in my spiritual life because as the choir grows I am also growing spiritually, said Ms Banda"


In [672]:
# Use BERT to insert equivalent words

def Augment(sentence, n):
    augmented_temp = []
    TOPK = 20 #default=100
    ACT = 'insert' #"substitute"
 
    aug_bert = naw.ContextualWordEmbsAug(model_path='distilbert-base-uncased', 
                                         device='cuda', 
                                         action = ACT, 
                                         top_k = TOPK)
    for ii in range(n):
        augmented_text = aug_bert.augment(sentence)
        augmented_temp.append(augmented_text)
    
    return augmented_temp

In [673]:
english_augs = []
for i, sentence in enumerate(augment_sents['orig sent']):
#     print(Augment(sentence))
#     augment_sents.iloc[i]['English Augmentations'] = (Augment(sentence))
    english_augs.append(Augment(sentence, 1))

In [674]:
# Flatten the list
english_augs = [item for sublist in english_augs for item in sublist]

In [675]:
english_augs_nb = []
for string in english_augs:
    if (string!=""):
        english_augs_nb.append(string)

In [676]:
len(english_augs_nb)

1463

In [677]:
label_aug = pd.DataFrame(columns=['english augmentations','chichewa translation', 'label'])

In [678]:
# The dataframe holding the English augmentations and their Chichewa translations:
label_aug['english augmentations'] = english_augs_nb

In [679]:
# Translate the augmented sentences to Chichewa
translated_temp = []
for i, sentence in enumerate(label_aug['english augmentations']):
    print(sentence)
    result = translator.translate(sentence, src = 'en', dest = 'ny')
    print(result.text)
    translated_temp.append(result.text)
label_aug['chichewa translation'] = translated_temp
label_aug['label'] = 'RELIGION'

abbey st
abbey st
mary'church s cathedral choir founder james chilomoni celebrates 40 years outstanding writer : mark sylvester media customer website wp - content / media uploads / 2019 / 09 09 / 2012 chilomoni
mary'church's cathedral choir founder james chilomoni akondwerera zaka 40 wolemba bwino kwambiri
print jpg " 0 alt = " " x width = " 0 427 " x height = " − 391 " / > 1000 st
sindikizani jpg " 0 alt = " " x wide = " 0 427 " x kutalika = " - 391 " / > 1000 st
james st chilomoni church located in glen blantyre black st
james st chilomoni church ili mu glen blantyre black st
john's choir master james macleod of montrose the lormon parish in aberdeen the archdiocese town of blantyre has commended on his long - tenure serving members of the choir for delivering their dedicated work and in promoting the sacred choir
john's choir master james macleod waku montrose parishi ya lormon ku aberdeen arkdayosizi ya blantyre wayamikira kamba ka nthawi yomwe wakhala akutumikira a kwayayi kamba 

pakadali pano chipanichi chili ndi likulu lake osati ku Italy komweko koma lili m'maiko ena osiyanasiyana
pope clement prays for heavenly peace – writer : thank heavens you chapola
Papa clement akupempherera mtendere wakumwamba - writer : thank heavens you chapola
jpg format 308w " sizes = " ( pdf max - width : × 300px ) 100vw, × 300px " / > pope : because enmity brings open war it only leads japan to open war, never means to cease
jpg mtundu 308w "sizes = " ( pdf max - wide : × 300px ) 100vw, × 300px " /> papa : chifukwa udani umabweretsa nkhondo yotseguka umangotsogolera Japan kutsegula nkhondo, sizitanthauza kuti asiye
the papal pope addressed this wednesday reception to a crowd that gathered there at st
papa analankhula pa madyerero a lachitatu limeneli kwa khamu la anthu limene linasonkhana kumeneko ku St
paul's s s square
malo a paul
st peters memorial square at the episcopal church'its s headquarters in the vatican
St peters memorial Square ku likulu la mpingo wa Episcopal Churc

Nthawi zina amanena kuti munthu wopembedza amene amapita kukaona malo opatulikawa ayenera kukhala bwino - wokonzeka kuti apeze ufulu wambiri.
during the ceremony, the uganda veterans association donated just two batteries to radio mt maria in malawi and, according reports to the listener group's executive chairperson, grace alexandra kalanda, privately thanked the station for enhancing its coverage of during the ugandan tour
pamwambowo bungwe la akale mu uganda lapereka mabatire awiri okha ku wailesi ya mt maria m’dziko la malawi ndipo malinga ndi malipoti a wapampando wamkulu wa gulu la omvera Grace alexandra kalanda anathokoza wailesiyi mwamseri kaamba kolimbikitsa kuululika kwa wailesiyi pa ulendo wa mdziko la Uganda.
also in making a statement, radio louis phiri malawi's managing program director louis paul phiri commended the organization for its commitment as to assisting the radio station in satisfying its programming needs
polankhulapo mkulu woyang’anira ma pologalamu ku radio 

ati sakugwiritsa ntchito njira zosiyanasiyana zoulutsira nkhani pofuna kukopa achinyamata a lgbt monga kufalitsa nsembe zaunyinji zachipembedzo pa intaneti mawayilesi kuti achinyamata asangalale ndi mapemphero awo.
2015 holy mother spirit church construction begins author : glory mission kondowe holy spirit community church congregation members 1 branch of south mzama parish in south dedza diocese members have also begun work on establishing a new church
Ntchito yomanga mpingo wa Holy Mother Spirit ya 2015 yayamba wolemba : ulemerero mission kondowe mpingo wa mpingo wa holy Spirit community
... according to church chairman edmond kamwana, ” the senate decision reportedly follows a growing number of protestant church clergy members
... malinga ndi kunena kwa tcheyamani wa tchalitchi cha edmond kamwana, ” chigamulo cha senate akuti chikutsatira chiŵerengero chokulirakulira cha atsogoleri achipembedzo achiprotestanti.
nowadays they say that right now because on the fifth day out of prayer

“Tikudziwa kuti mliri wa covid-fm 19 wabweretsa mavuto azachuma m’dziko muno koma atsimikiza mtima kuthandiza wayilesiyi bwinobwino ndipo ichi chikhale phunziro kwa ma parishi ena onse kuti asamangodzudzula chifukwa chothandiza anthu osaukawa. wayilesi,” adatero phiri
at attending the ceremony the ancient christians of monte st
pamwambowu akhristu akale aku monte St
marys church parish, pa ntaja has donated over more than 3 hundred and almost 6 thousand lei kwacha shillings to purchase the station
Marys church parishi, pa ntaja apereka ndalama zoposera 3 hundred and 6 thousand lei kwacha kuti agule stationyi.
pope commends marie vincent de de paul nuns the leader archbishop of the catholic catholic church worldwide, pope stephen francis, and has praised the nuns cause of conception st
Papa wayamikira marie vincent de paul nuns mtsogoleri wamkulu wa mpingo wakatolika pa dziko lonse Papa Stephen francis, ndipo wayamikira masisitere omwe anayambitsa kutenga mimba kwa St.
vincent leo de s 

...

Sunday Papa clement ali ndi misa yoyamikila kukumbukila ulendo wa Pasaka pa chisumbu cha mount lampedusa pamodzi ndi mtsogoleri wa mpingo wakatolika pa dziko lonse Papa francisco lachinayi likudzali lachitatu papa apanga misa lamulungu lokumbukila ulendo wake ku chilumba cha lampedusa
• according to vatican radio official reports, honoring the pope john will perform off the mass on july 28 8, 2020, commemorating the seven - consecutive year diplomatic visit to the island of donna lampedusa in mainland italy
• malinga ndi malipoti a wailesi ya vatican, kulemekeza Papa john adzayimba misa pa 28 july, 2020, kukumbukira ulendo wa zaka zisanu ndi ziwiri zotsatizana ku chilumba cha donna lampedusa ku mainland ku Italy.
arriving a few months ago after giving his election as presiding pope emeritus on march 9 13, 2013, pope francis visited la lampedusa abbey in italy on thursday july 8, 2013
akufika miyezi ingapo yapitayo atapereka chisankho chake ngati wotsogolera papa wotuluka pa Marichi 9 1

tikanangosowa chochita,' anadandaula motero
kapalasa wants the state government to intervene here as soon as be possible so ensure that farmers can get the objective most pointed out results of their production
kapalasa akufuna kuti boma lilowererepo pano mwachangu kuti alimi apeze zotsatira zomwe zadziwika bwino za ulimi wawo.
so the grandmother thinks that forming groups effectively will only help enable the farmers grow to reap the benefits ahead of farming
choncho gogo akuganiza kuti kupanga magulu mogwira mtima kungathandize alimi kuti akule ndikupeza phindu patsogolo pa ulimi.
many subsistence farmers working in mulanje rural district have harvested a lot of peas and so one is now still in the fields and waiting constantly for them starting to dry properly
alimi ambiri omwe amagwira ntchito m’boma la mulanje akumudzi akolola nandolo zambiri choncho wina akadali m’minda ndipo amadikirira nthawi zonse kuti ayambe kuwuma bwino.
less recently the head of the nandolo association organ

parishiyi ili ndi zosungirako zosungirako zosungirako zachilengedwe zochepera 75 mpaka pano gulu la abambo lakwanitsa kuyendera parishi iliyonse kwina ku Australia kuphatikiza ndi ena omwe akupereka maphunziro osamalira zachilengedwe.
radio maria urges its listeners not to fully give itself up in the upcoming season coronavirus radio maria malawi has urged its listeners too not to give up on the main station themselves and any support they may have, so that its operations can simply continue yet to thrive as operation covid - 19 has stopped many activities in outside the country
Radio maria yalimbikitsa omvera ake kuti asadzipereke kwathunthu mu season yomwe ikubwerayi ya coronavirus maria malawi yalimbikitsa omvera ake kuti nawonso asagonje pa wailesiyi ndi thandizo lililonse lomwe angakhale nalo, kuti ntchito zake zipitilirebe bwino. monga operation covid - 19 yayimitsa ntchito zambiri kunja kwa dziko
hill : coronavirus suspends almost all preparations he has said preparations for ma

pafupifupi ndinene apa tsopano kuti wotonthoza ubweya analibe wake
" at first seeing brenda lee was talking up to my brother and just then i knew i had nothing to actually do something, " said wanda faith
"Poyamba ndikuwona brenda lee akulankhula ndi mchimwene wanga ndipo nthawi yomweyo ndidazindikira kuti palibe chomwe ndingachite," adatero wanda Faith.
very gradually the two formed a conversation and faith healer did not hesitate individually but dropped the words that christ would instantly unite all the two as having a family
pang'onopang'ono awiriwa anayamba kucheza ndipo sing'anga sanazengereze aliyense payekha koma anagwetsa mawu oti khristu nthawi yomweyo alumikizane onse awiri kukhala ndi banja.
after a while there was silence, we did not immediately know the answer but on august 28, 2009 i received back a anonymous phone call telling me that he had agreed along with what i had told him, i said holy faith, probably the second girl born in ireland a catholic family of three boy

odzipereka a presbyterian anayamba kumvetsera ulaliki wa Mr
chimbalanga father - in - chief of christ the parish ii parish in mangochi diocese father john saulos of chimbalanga said this incident on saturday prayers in neighbouring balaka district court after a protest led by ministers serving the station in just all three parts of the country
bambo a chimbalanga - in - chief of christ parish ii parish mu dayosizi ya mangochi bambo john saulos a chimbalanga ndi omwe anena izi pa mapemphero a loweruka m’bwalo loyandikana ndi bwalo la balaka pambuyo pa zionetsero zomwe nduna zotumikira wailesiyi m’magawo atatu okha a dziko lino.
they say that the ministers of the true gospel must all be made the first ones to benefit greatly from the abundant good news they preach
amanena kuti atumiki a uthenga wabwino woona ayenera kukhala oyamba kupindula kwambiri ndi uthenga wabwino wochuluka umene amalalikira.
the volunteers have done a great responsibility here to spread the message ahead of god who

In [680]:
label_aug

,english augmentations,chichewa translation,label
0,abbey st,abbey st,RELIGION
1,mary'church s cathedral choir founder james chilomoni celebrates 40 years outstanding writer : mark sylvester media customer website wp - content / media uploads / 2019 / 09 09 / 2012 chilomoni,mary'church's cathedral choir founder james chilomoni akondwerera zaka 40 wolemba bwino kwambiri,RELIGION
2,"print jpg "" 0 alt = "" "" x width = "" 0 427 "" x height = "" − 391 "" / > 1000 st","sindikizani jpg "" 0 alt = "" "" x wide = "" 0 427 "" x kutalika = "" - 391 "" / > 1000 st",RELIGION
3,james st chilomoni church located in glen blantyre black st,james st chilomoni church ili mu glen blantyre black st,RELIGION
4,john's choir master james macleod of montrose the lormon parish in aberdeen the archdiocese town of blantyre has commended on his long - tenure serving members of the choir for delivering their dedicated work and in promoting the sacred choir,john's choir master james macleod waku montrose parishi ya lormon ku aberdeen arkdayosizi ya blantyre wayamikira kamba ka nthawi yomwe wakhala akutumikira a kwayayi kamba kopereka ntchito zawo zodzipereka komanso kulimbikitsa kwaya yopatulika.,RELIGION
...,...,...,...
1458,"the volunteers have done a great responsibility here to spread the message ahead of god who mankind is holy so they all too should be officially sanctified, recently said mr cyril chimbalanga","anthu odziperekawa apanga udindo waukulu kuno kufalitsa uthenga pamaso pa mulungu yemwe anthu ndi oyera kotero nawonso ayeretsedwe mwalamulo, posachedwapa atero Mr cyril chimbalanga.",RELIGION
1459,"speaking on of the occasion, joseph kimu, director elect of establishing the radio station, said he still was pleased greatly with improving the performance qualities of establishing the station",Polankhulapo mkulu wosankhidwa wokhazikitsa wayilesiyi joseph kimu adati akadali okondwa kwambiri ndikuchita bwino pakukhazikitsa wailesiyi.,RELIGION
1460,"he encouraged many us to love prayer and pray to help do what we gotta tell these people needed to do, said mr","adatilimbikitsa ambiri kuti tizikonda mapemphero ndikupemphera kuti atithandize kuchita zomwe tiyenera kuuza anthuwa, atero a Mr",RELIGION
1461,kim kimu,kim kimu,RELIGION


In [681]:
label_aug.to_csv('./religion.csv')

In [682]:
del label_aug

In [683]:
flooding = pd.read_csv('./flooding.csv')
arts_crafts = pd.read_csv('./ArtsCrafts.csv')
transport = pd.read_csv('./transport.csv')
music = transport = pd.read_csv('./music.csv')
witchcraft = pd.read_csv('./witchcraft.csv')
culture = pd.read_csv('./culture.csv')
localchiefs = pd.read_csv('./localchiefs.csv')
opinion_essay = pd.read_csv('./OpinionEssay.csv')
wild_env = pd.read_csv('./WildEnviron.csv')
relationships = pd.read_csv('./relation.csv')
educ = pd.read_csv('./education.csv')
sports = pd.read_csv('./sports.csv')
farming = pd.read_csv('./farming.csv')
economy = pd.read_csv('./economy.csv')
health = pd.read_csv('./health.csv')
social_issues = pd.read_csv('./SocialIssues.csv')
law_order = pd.read_csv('./LawOrder.csv')
religion = pd.read_csv('./religion.csv')
social = pd.read_csv('./social.csv')
politics = pd.read_csv('./politics.csv')

In [685]:
frames = [flooding, arts_crafts, transport, music, witchcraft, culture, localchiefs, opinion_essay, wild_env, 
          relationships, educ, sports, farming, economy, health, social_issues, law_order, religion, social, politics]

In [687]:
Augmentations = pd.concat(frames, axis = 0)

In [688]:
Augmentations

,Unnamed: 0,english augmentations,chichewa translation,label
0,0,"work to prevent water borne hazards helps with raising the issue instead of rain forecast in the forefront, whereas the thai government bureaucracy is working hard to ensure awareness that the big floods that hit some major areas this year don't become a problem for you","ntchito yopewera ngozi zamadzi imathandizira kukweza nkhaniyi m'malo monena za mvula patsogolo, pomwe akuluakulu aboma la Thailand akuyesetsa kuwonetsetsa kuti kusefukira kwamadzi komwe kudawononga madera akuluakulu chaka chino kusakhale vuto kwa inu.",FLOODING
1,1,"work starts to properly prevent water hazards with the issue of rain forecast being in the forefront, hence the kerala government is already working hard here to ensure maintenance that all the floods that hit some areas this year do will not become a problem for you","ntchito yayamba bwino kupewa ngozi za madzi pomwe nkhani yolosera mvula ili patsogolo, ndiye kuti boma la kerala likulimbikira kale kuno kuonetsetsa kuti kusefukira kwa madzi osefukira mmadera ena chaka chino kusakhale vuto kwa inu.",FLOODING
2,2,"· work to prevent natural water hazards with eliminating the issue aspects of rain forecast being in the forefront, the government is continually working on hard actively to ensure that even the floods that hit some areas this year don't actually become a problem for you","· gwirani ntchito popewa ngozi za madzi achilengedwe pothetsa nkhani zonena kuti mvula ikhala patsogolo, boma likulimbikira mosalekeza kuti ngakhale kusefukira kwa madzi komwe kunachitika m’madera ena chaka chino kusakhale vuto kwa inu.",FLOODING
3,3,"work also to prevent water hazards with the pressing issue of rain forecast issues in around the indian forefront, the government is actively working hard to ensure that the massive floods that hit some areas like this dry year don't become a problem for everyday you","gwirani ntchito kuti mupewe ngozi zamadzi chifukwa chazovuta zanyengo zamtsogolo za mvula kuzungulira India, boma likugwira ntchito molimbika kuwonetsetsa kuti kusefukira kwamadzi komwe kumakhudza madera ena ngati chaka chouma chisakhale vuto tsiku lililonse.",FLOODING
4,4,"work to prevent water hazards dealt with the issue of rain hazards forecast in the delhi forefront, the government is actively working hard to ensure ensuring that stopping the floods present that hit some areas this year should don't become serious a problem even for you","ntchito yopewera ngozi za madzi idathana ndi nkhani yolosera za ngozi za mvula ku delhi kutsogolo, boma likugwira ntchito molimbika kuwonetsetsa kuti kuletsa kusefukira komwe kwakhudza madera ena chaka chino kusakhale vuto lalikulu ngakhale kwa inu.",FLOODING
...,...,...,...,...
5076,5076,"when the newspaper interviewed a 122 of of the nominated 193 members, 75 councillors said otherwise they would reject the idea, quoting 17 elected mps which had not yet decided whether whether to accept it or not while 30 members officially said they would agree to it","Pamene nyuzipepalayi idacheza ndi mamembala 122 mwa 193 omwe adasankhidwa, makhansala 75 adati akana ganizoli, ponena za aphungu 17 omwe adasankhidwa omwe sanasankhepo kuti avomereze kapena ayi pomwe mamembala 30 adati avomereza.",POLITICS
5077,5077,"in april 2006, president peter mutharika said that such a joint government project could contribute substantially to accelerating the country'’ s development","mu April 2006, pulezidenti peter mutharika adanena kuti ntchito yogwirizana ndi boma ingathandize kwambiri kuti chitukuko cha dziko chifulumire.",POLITICS
5078,5078,since this was basically a meeting to review the laws adoption of laws this country,popeza uwu unali msonkhano wowunikanso malamulo otengera malamulo adziko lino,POLITICS
5079,5079,"but for today, mutharika opposes rejects the indian idea of land partition",koma lero mutharika akutsutsa ganizo la mmwenye logawa malo,POLITICS


In [689]:
Augmentations.to_csv('./Train_Augmented')